# Développez un chatbot pour réserver des vacances

<img src='https://user.oc-static.com/upload/2019/10/24/15719199766568_Capture%20d%E2%80%99e%CC%81cran%202019-10-24%20a%CC%80%2014.14.41.png'>


Fly Me est une agence qui propose des voyages clé en main pour les particuliers ou les professionnels. 
 

Fly Me a lancé un projet ambitieux de développement d’un chatbot pour aider les utilisateurs à choisir une offre de voyage.

La première étape de ce projet est de construire un MVP qui aidera les employés de Fly Me à réserver facilement un billet d’avion pour leurs vacances.

Comme ce projet est itératif, nous avons limité les fonctionnalités de la V1 du chatbot. La V1 devra pouvoir identifier dans la demande de l’utilisateur les cinq éléments suivants :

Ville de départ
Ville de destination
Date aller souhaitée du vol
Date retour souhaitée du vol
Budget maximum pour le prix total des billets

# Observation du jeu de données

In [335]:
import pandas as pd
import numpy as np
import json

with open('../frames (1)/frames.json') as f:
    df = json.load(f)
    
    
print(f'Le jeu de données contient {len(df)} conversations entre bot & utilisateur')

df[0]

Le jeu de données contient 1369 conversations entre bot & utilisateur


{'user_id': 'U22HTHYNP',
 'turns': [{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
   'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
      'name': 'inform'},
     {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
       {'val': 'Caprica', 'key': 'or_city'},
       {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
       {'val': '8', 'key': 'n_adults'},
       {'val': '1700', 'key': 'budget'}],
      'name': 'inform'}],
    'active_frame': 1,
    'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
 

In [336]:
df[0].keys()

dict_keys(['user_id', 'turns', 'wizard_id', 'id', 'labels'])

In [341]:
df[0]['user_id'], df[0]['wizard_id'], df[0]['id'], df[0]['id']

('U22HTHYNP',
 'U21DKG18C',
 'e2c0fc6c-2134-4891-8353-ef16d8412c9a',
 'e2c0fc6c-2134-4891-8353-ef16d8412c9a')

In [342]:
df[1]['user_id'], df[1]['wizard_id'], df[1]['id'], df[1]['id']

('U21E41CQP',
 'U21DMV0KA',
 '4a3bfa39-2c22-42c8-8694-32b4e34415e9',
 '4a3bfa39-2c22-42c8-8694-32b4e34415e9')

In [302]:
df[0]['turns']

[{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
     'name': 'inform'},
    {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
      {'val': 'Caprica', 'key': 'or_city'},
      {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
      {'val': '8', 'key': 'n_adults'},
      {'val': '1700', 'key': 'budget'}],
     'name': 'inform'}],
   'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
     'name': 'inform'},
    {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
      {'val': 'Caprica', 'key': 'or_city'},
      {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
      {'val': '8', 'key': 'n_adults'},
      {'val': '1700', 'key': 'budget'}],
     'name': 'inform'}],
   'active_frame': 1,
   'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
      'budget': [{'val': '1700.0', 'negated': False}],

In [346]:
print(f"La première conversation contient {len(df[0]['turns'])} échanges.")

[(u['author'],u['text']) for u in df[0]['turns']]

La première conversation contient 7 échanges


[('user',
  "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700."),
 ('wizard',
  'Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?'),
 ('user',
  'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.'),
 ('wizard',
  'I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates?'),
 ('user',
  'I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?'),
 ('wizard',
  'I checked the availability for that date and there were no trips available.  Would you like to select some alternate dates?'),
 ('user',
  "I suppose I'll speak with my husband to see if we can choose other dates, and then I'll come back to you.Thanks for your he

# Data préparation

Nous préparons nos données afin que notre algoritme reconnaisse les entités suivante :
- Ville de départ --> or_city
- Ville de destination --> dst_city
- Date aller souhaitée du vol --> str_date
- Date retour souhaitée du vol --> end_date
- Budget maximum pour le prix total des billets --> budget

In [16]:
mlEntityDefinition = ['or_city', 'dst_city','str_date','end_date','budget']

In [356]:
def collect_data(df,n):
    # Création d'une liste des textes accompagnés de leurs entités 
    base_data =[]
    
    # Sélection de n conversation du jeu de données
    for i in range(n):
        # Filtrage des réponses du bot
        data = [u for u in df[i]['turns'] if 'db' not in u.keys()]
        
        # Récupération des messages utilisateur avec leurs entités
        for messages in data: 
            text = messages['text']
            info = messages['labels']['frames'][-1]['info']
            [base_data.append((text, u,info[u][0]['val'])) for u in info.keys() if u in mlEntityDefinition]
     
    # Création d'un dataframe contenant le text l'entité et la valeur de l'entité
    base_data = pd.DataFrame(base_data, columns=['text', 'entit', 'val'])
    base_data.val = base_data.val.apply(lambda x: x.split('.')[0])
    
    # Filtre permettant de ne conserver uniquement les entité présent dans le text
    filtre = [base_data.val.values[n] in base_data.text.values[n] for n in range(len(base_data))]
    
    return base_data[filtre]

# Exemble de jeu de données pour les 8 premières conversions
test = collect_data(df, 3)
test

,text,entit,val
0,I'd like to book a trip to Atlantis from Capri...,budget,1700
1,I'd like to book a trip to Atlantis from Capri...,dst_city,Atlantis
2,I'd like to book a trip to Atlantis from Capri...,or_city,Caprica
4,"Yes, how about going to Neverland from Caprica...",budget,1900
5,"Yes, how about going to Neverland from Caprica...",dst_city,Neverland
6,"Yes, how about going to Neverland from Caprica...",or_city,Caprica
9,I have no flexibility for dates... but I can l...,dst_city,Atlantis
10,I have no flexibility for dates... but I can l...,or_city,Atlantis
16,"Hello, I am looking to book a vacation from Go...",or_city,Gotham City
17,"Hello, I am looking to book a vacation from Go...",dst_city,Mos Eisley


In [357]:
# Fonction permettant de localiser les entités dans le text
def get_example_label(utterance, entity_name, value):
    """Build a EntityLabelObject.
    This will find the "value" start/end index in "utterance", and assign it to "entity name"
    """
    utterance = utterance.lower()
    value = value.lower()
    return {
        'entity_name': entity_name,
        'start_char_index': utterance.find(value),
        'end_char_index': utterance.find(value) + len(value)
    }

# Exemple de sortie
[get_example_label(test.text.iloc[n], test.entit.iloc[n], test.val.iloc[n]) for n in range(len(test))]

[{'entity_name': 'budget', 'start_char_index': 117, 'end_char_index': 121},
 {'entity_name': 'dst_city', 'start_char_index': 27, 'end_char_index': 35},
 {'entity_name': 'or_city', 'start_char_index': 41, 'end_char_index': 48},
 {'entity_name': 'budget', 'start_char_index': 114, 'end_char_index': 118},
 {'entity_name': 'dst_city', 'start_char_index': 24, 'end_char_index': 33},
 {'entity_name': 'or_city', 'start_char_index': 39, 'end_char_index': 46},
 {'entity_name': 'dst_city', 'start_char_index': 56, 'end_char_index': 64},
 {'entity_name': 'or_city', 'start_char_index': 56, 'end_char_index': 64},
 {'entity_name': 'or_city', 'start_char_index': 44, 'end_char_index': 55},
 {'entity_name': 'dst_city', 'start_char_index': 59, 'end_char_index': 69},
 {'entity_name': 'budget', 'start_char_index': 75, 'end_char_index': 79},
 {'entity_name': 'or_city', 'start_char_index': 23, 'end_char_index': 34},
 {'entity_name': 'dst_city', 'start_char_index': 38, 'end_char_index': 47},
 {'entity_name': 'd

In [362]:
# Fonction permettant de sortir le jeu de données sous format json en vu d'interpretation par le model Luois
def labeledExampleUtterance(data, intentName = "FlyMe.Booking"):
    
    data_Uterance = []
    for text in data.text.unique():
        data_temp = data[data.text == text][['entit','val']]
        data_json = {
            "text": text,
            "intentName": intentName,
            "entityLabels": [get_example_label(text, data_temp.entit.iloc[n], data_temp.val.iloc[n]) 
                             for n in range(len(data_temp))]
        }
        data_Uterance.append(data_json)
        
    return data_Uterance


# Exemple de sortie de fonction

labeledExampleUtteranceWithMLEntity = labeledExampleUtterance(test)
labeledExampleUtteranceWithMLEntity

[{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  'intentName': 'FlyMe.Booking',
  'entityLabels': [{'entity_name': 'budget',
    'start_char_index': 117,
    'end_char_index': 121},
   {'entity_name': 'dst_city', 'start_char_index': 27, 'end_char_index': 35},
   {'entity_name': 'or_city', 'start_char_index': 41, 'end_char_index': 48}]},
 {'text': 'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.',
  'intentName': 'FlyMe.Booking',
  'entityLabels': [{'entity_name': 'budget',
    'start_char_index': 114,
    'end_char_index': 118},
   {'entity_name': 'dst_city', 'start_char_index': 24, 'end_char_index': 33},
   {'entity_name': 'or_city', 'start_char_index': 39, 'end_char_index': 46}]},
 {'text': 'I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?',
  'intentName': 'FlyMe.Booking',

# Création du Bot Fly Me 

In [360]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

authoringKey = '123007a9fbfb4ec2bd2e4cd7c88c37fa'
authoringEndpoint = 'https://languep10-authoring.cognitiveservices.azure.com/'
predictionKey = '75800c252f4246e9b665a9015f046091'
predictionEndpoint = 'https://langue-p10.cognitiveservices.azure.com/'


# We use a UUID to avoid name collisions.
appName = "Fly Me" + str(uuid.uuid4())
versionId = "0.1"
intentName = "FlyMe.Booking"



client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

# define app basics
appDefinition = ApplicationCreateObject(name=appName, initial_version_id=versionId, culture='en-us')

# create app
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))


client.model.add_intent(app_id, versionId, intentName)

Created LUIS app with ID 6c580a9c-2c97-4173-8a7e-ebc60b93ee2c


'4cb77641-dd46-4f68-a3b6-e04b32a8686f'

# Création des intentions


In [502]:
client.model.add_entity(app_id, versionId, name='end_date')

ErrorResponseException: Operation returned an invalid status code 'Bad Request'

In [501]:
mlEntityDefinition = ['or_city', 'dst_city','str_date','end_date','budget']

[client.model.add_entity(app_id, versionId, name=Entity) for Entity in mlEntityDefinition]

ErrorResponseException: Operation returned an invalid status code 'Bad Request'

# Envoi des examples de phrases avec entité

In [470]:
# Define labeled example

# Add an example for the entity.
# Enable nested children to allow using multiple models with the same name.
# The quantity subentity and the phraselist could have the same exact name if this is set to True
[client.examples.add(app_id, versionId, example) for example in labeledExampleUtteranceWithMLEntity]

# Entrainement du modèle Luis

In [364]:
client.train.train_version(app_id, versionId)
waiting = True
while waiting:
    info = client.train.get_status(app_id, versionId)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


# Publication & appel du model

In [366]:
client.apps.update_settings(app_id, is_public=True)

responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)

responseEndpointInfo

# Prediction

In [394]:
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

# Production == slot name
predictionRequest = { "query" : "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700." }

predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest, show_all_intents=True)
print("Top intent: {}".format(predictionResponse.prediction.top_intent))
print("Sentiment: {}".format (predictionResponse.prediction.sentiment))
print("Intents: ")

for intent in predictionResponse.prediction.intents:
    print("\t{}".format (json.dumps (intent)))
print("Entities: {}".format (predictionResponse.prediction.entities))

Top intent: FlyMe.Booking
Sentiment: None
Intents: 
	"FlyMe.Booking"
	"None"
Entities: {'dst_city': ['Atlantis'], 'or_city': ['Caprica'], 'budget': ['1700.']}


# Scoring 

In [559]:
data = collect_data(df, 80)
data

,text,entit,val
0,I'd like to book a trip to Atlantis from Capri...,budget,1700
1,I'd like to book a trip to Atlantis from Capri...,dst_city,Atlantis
2,I'd like to book a trip to Atlantis from Capri...,or_city,Caprica
4,"Yes, how about going to Neverland from Caprica...",budget,1900
5,"Yes, how about going to Neverland from Caprica...",dst_city,Neverland
...,...,...,...
2293,Anything going to Caprica,dst_city,Caprica
2295,"Good afternoon, my man! I'd like to plan a hon...",or_city,Madrid
2296,"Good afternoon, my man! I'd like to plan a hon...",dst_city,Naples
2309,Hi i want to plan my honeymoon to rome,dst_city,rome


In [560]:
data_json = labeledExampleUtterance(data)
data_json

[{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  'intentName': 'FlyMe.Booking',
  'entityLabels': [{'entity_name': 'budget',
    'start_char_index': 117,
    'end_char_index': 121},
   {'entity_name': 'dst_city', 'start_char_index': 27, 'end_char_index': 35},
   {'entity_name': 'or_city', 'start_char_index': 41, 'end_char_index': 48}]},
 {'text': 'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.',
  'intentName': 'FlyMe.Booking',
  'entityLabels': [{'entity_name': 'budget',
    'start_char_index': 114,
    'end_char_index': 118},
   {'entity_name': 'dst_city', 'start_char_index': 24, 'end_char_index': 33},
   {'entity_name': 'or_city', 'start_char_index': 39, 'end_char_index': 46}]},
 {'text': 'I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?',
  'intentName': 'FlyMe.Booking',

In [561]:
# Séparation en jeu d'entrainement et test
from sklearn.model_selection import train_test_split

X_train , X_test = train_test_split(data_json)

len(X_train), len(X_test)

(164, 55)

In [565]:
# Publication des phrases example de X
for example in X_train:
    try:
        client.examples.add(app_id, versionId, example)
    except :
        pass

In [566]:
# Entrainement
client.train.train_version(app_id, versionId)
waiting = True
while waiting:
    info = client.train.get_status(app_id, versionId)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False
        
# Publication       
client.apps.update_settings(app_id, is_public=True)
responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)

Waiting 10 seconds for training to complete...
trained


In [568]:
def luis_score(X):
    score = []
    for x in X:
        predictionRequest = { "query" : x['text']}
        
        predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, 
                                                                          "Production", 
                                                                          predictionRequest, 
                                                                          show_all_intents=True)
        
        score.append(predictionResponse.prediction.intents['FlyMe.Booking'].score)
    return np.array(score).mean()

In [569]:
luis_score(X_train)

0.998857898109756

In [570]:
luis_score(X_test)

0.9814074969345454

# Création d'un pipline de transformation

In [546]:
class My_bot():
    def __init__(self, name):
        self.authoringKey = '123007a9fbfb4ec2bd2e4cd7c88c37fa'
        self.authoringEndpoint = 'https://languep10-authoring.cognitiveservices.azure.com/'
        self.predictionKey = '75800c252f4246e9b665a9015f046091'
        self.predictionEndpoint = 'https://langue-p10.cognitiveservices.azure.com/'
        
        # We use a UUID to avoid name collisions.
        self.appName = name + str(uuid.uuid4())
        self.versionId = "0.1"
        self.intentName = "FlyMe.Booking"
        
        self.client = LUISAuthoringClient(self.authoringEndpoint, 
                                          CognitiveServicesCredentials(self.authoringKey))

        # define app basics
        self.appDefinition = ApplicationCreateObject(name=self.appName, 
                                                     initial_version_id=self.versionId, culture='en-us')

        # create app
        self.app_id = self.client.apps.add(self.appDefinition)

        # get app id - necessary for all other changes
        print("Created LUIS app with ID {}".format(self.app_id))


        self.client.model.add_intent(self.app_id, self.versionId, self.intentName)

    def add_entity(self, mlEntityDefinition = ['or_city', 'dst_city','str_date','end_date','budget']):
        [self.client.model.add_entity(self.app_id, self.versionId, name=Entity) 
         for Entity in mlEntityDefinition]
        
        
        
    def collect_data(self, df, n):
        # Création d'une liste des textes accompagnés de leurs entités 
        base_data =[]

        # Sélection de n conversation du jeu de données
        for i in range(n):
            # Filtrage des réponses du bot
            data = [u for u in df[i]['turns'] if 'db' not in u.keys()]

            # Récupération des messages utilisateur avec leurs entités
            for messages in data: 
                text = messages['text']
                info = messages['labels']['frames'][-1]['info']
                [base_data.append((text, u,info[u][0]['val'])) 
                 for u in info.keys() if u in mlEntityDefinition]

        # Création d'un dataframe contenant le text l'entité et la valeur de l'entité
        base_data = pd.DataFrame(base_data, columns=['text', 'entit', 'val'])
        base_data.val = base_data.val.apply(lambda x: x.split('.')[0])

        # Filtre permettant de ne conserver uniquement les entité présent dans le text
        filtre = [base_data.val.values[n] in base_data.text.values[n] for n in range(len(base_data))]

        self.base_data = base_data[filtre]

        return base_data[filtre]

    # Fonction permettant de sortir le jeu de données sous format 
    #json en vu d'interpretation par le model Luois
    def labeledExampleUtterance(self, data, intentName = "FlyMe.Booking"):

        data = self.base_data
        def get_example_label(utterance, entity_name, value):
            """Build a EntityLabelObject.
            This will find the "value" start/end index in "utterance", and assign it to "entity name"
            """
            utterance = utterance.lower()
            value = value.lower()
            return {
                'entity_name': entity_name,
                'start_char_index': utterance.find(value),
                'end_char_index': utterance.find(value) + len(value)
            }

        data_Uterance = []
        for text in data.text.unique():
            data_temp = data[data.text == text][['entit','val']]
            data_json = {
                "text": text,
                "intentName": intentName,
                "entityLabels": [get_example_label(text, data_temp.entit.iloc[n], data_temp.val.iloc[n]) 
                                 for n in range(len(data_temp))]
            }
            data_Uterance.append(data_json)
            
        self.utterance = data_Uterance

        for example in  self.utterance:
            try :
                client.examples.add(self.app_id, self.versionId, example)
            except : pass
        print('Example published')
        
        
    def train(self):
        self.client.train.train_version(self.app_id, self.versionId)
        waiting = True
        while waiting:
            info = self.client.train.get_status(self.app_id, self.versionId)

            # get_status returns a list of training statuses, one for each model. 
            #Loop through them and make sure all are done.
            waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, 
                              info))
            if waiting:
                print ("Waiting 10 seconds for training to complete...")
                time.sleep(10)
            else: 
                print ("trained")
                waiting = False
                
        self.client.apps.update_settings(self.app_id, is_public=True)
        self.responseEndpointInfo = self.client.apps.publish(self.app_id, versionId, is_staging=False)
        
        
        
    def prediction(self, text):
        self.runtimeCredentials = CognitiveServicesCredentials(predictionKey)
        self.clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

        # Production == slot name
        self.predictionRequest = { "query" : text}

        self.predictionResponse = self.clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest, show_all_intents=True)
        print("Top intent: {}".format(predictionResponse.prediction.top_intent))
        print("Sentiment: {}".format (predictionResponse.prediction.sentiment))
        print("Intents: ")

        for intent in self.predictionResponse.prediction.intents:
            print("\t{}".format (json.dumps (intent)))
        print("Entities: {}".format (predictionResponse.prediction.entities))
        
        return self.predictionResponse.prediction.intents
        
    
    def luis_score(X):
        score = []
        for x in X:
            predictionRequest = { "query" : x['text']}

            predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, 
                                                                              "Production", 
                                                                              predictionRequest, 
                                                                              show_all_intents=True)

            score.append(predictionResponse.prediction.intents['FlyMe.Booking'].score)
        return np.array(score).mean()

In [547]:
bot = My_bot('test3')

Created LUIS app with ID 67f94d39-f109-4f68-b962-00c7b39ec2e5


In [550]:
bot.labeledExampleUtterance(1)